# Langevin integrator with WCA potential

Simple implementation of two particles following Langevin dynamics

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import msmrd2
import msmrd2.visualization as msmrdvis
from msmrd2.integrators import langevin 
from msmrd2.potentials import WCA

In [ ]:
# Define arbitrarily large list of particles
np.random.seed(seed=1)
numparticles = 500
boxsize = 25
D = 0.1
mass = 1.0
velocity = np.zeros(3)
pyPartlist = [] 
for i in range(numparticles):
    overlap = True
    while overlap:
        position = np.array([boxsize*np.random.rand()-0.5*boxsize,
                             boxsize*np.random.rand()-0.5*boxsize,
                             boxsize*np.random.rand()-0.5*boxsize])
        overlap = False     
        for j in range(len(pyPartlist)):
            if np.linalg.norm(position - pyPartlist[j].position) < 2.0:
                overlap = True
                continue
                
    part = msmrd2.particle(D, position, velocity, mass)
    pyPartlist.append(part)

In [ ]:
# Create list of particles that can be read from msmrd
# note the particles in this list will be independent from the python list.
partlist = msmrd2.integrators.particleList(pyPartlist)

In [ ]:
# Over-damped Langevin integrator definition
dt = 0.0005
seed = -1 # Seed = -1 used random device as seed
bodytype = 'point'
integrator = langevin(dt, seed, bodytype) 

In [ ]:
# Define boundary (choose either spherical or box)
boxBoundary = msmrd2.box(boxsize,boxsize,boxsize,'periodic')
#boxBoundary = msmrd2.box(boxsize,boxsize,boxsize,'reflective')
integrator.setBoundary(boxBoundary)

In [ ]:
# Define Lennard-Jones potential (rm=2^(1/6)sigma)
epsilon = 1.0
rm = 0.5
sigma = rm * 2**(-1/6)
potentialWCA = WCA(epsilon, sigma)
integrator.setPairPotential(potentialWCA)

In [ ]:
# Integrate the particles, save to .xyz to produce VMD output (additional overhead)
timesteps = 200000
stride = 20
datafile  = open('../../data/vmd/langevinWCA.xyz', 'w')
for i in range(timesteps):
    if i%stride == 0:
        datafile.write(str(len(partlist)) + '\n')
        datafile.write(str(0) + '\n')
    for j, part in enumerate(partlist):
        if i%stride == 0:
            v0 = part.position
            datafile.write('type_0' + ' ' + ' '.join(map(str, v0)) + '\n')
    integrator.integrate(partlist)
    if i%1000 == 0:
        print("Percentage complete: ", 100*i/timesteps, "%", end="\r")
datafile.close()
print("Percentage complete: ", 100, " %")

In [ ]:
# Generate TCL script to visualize with VMD
msmrdvis.generateTCL_sphericalParticles(numparticles = numparticles, outfname = "langevinWCA", 
                              tclfname = "../../data/vmd/langevinWCA_2vmd.tcl")

To load the movie go to /data/vmd and run in a terminal "vmd -e langevinLennardJones_2vmd.tcl".


## In case VMD is not desired

In case VMD output is not desired, below we simply integrate the model and show the output directly.

In [ ]:
# Integrate particle list and print only positions (prints positions and orientations of both particles)
timesteps = 100
print('{:<10s}{:<15s}{:<40s}{:<40s}'.format("Iteration", "Time", "Positions", "Velocites"))
for i in range(timesteps):
    print('{:<10d}{:<10f}{:<40s}{:<50s}'.format(i, integrator.clock, str(partlist[0].position), str(partlist[0].velocity)))
    print('{:<10s}{:<10s}{:<40s}{:<50s}'.format(str(" "), str(" "), str(partlist[1].position), str(partlist[1].velocity)))
    integrator.integrate(partlist)

# Calculate and plot potential

Plots the Lennard jones potential for a set of given parameters:

In [ ]:
# Redefine potential
epsilon = 1
rm = 1.0
sigma = rm * 2**(-1/6)
cutOff = 5
potentialWCA = WCA(epsilon, sigma)

# Construct evaluate function
def WCAevaluate(r):
    p1 = np.zeros(3)
    p2 = np.array([r,0,0])
    vel = np.zeros(3)
    part1 = msmrd2.particle(1.0, p1, vel, 1)
    part2 = msmrd2.particle(1.0, p2, vel, 1)
    return potentialWCA.evaluate(part1, part2)

rarray = np.linspace(0.1,5,100)
V_WCA = [WCAevaluate(r) for r in rarray]
plt.plot(rarray, V_WCA)
plt.xlabel("Distance r")
plt.ylabel("WCA potential")
plt.ylim([-1.5,3])
plt.show()